# Detect claims to fact check in political debates

In this project you will implement various classifiers using both neural and feature based technqiues to detect which sentences in political debates should be fact checked.
Dataset from ClaimBuster: https://zenodo.org/record/3609356 
Evaluate your classifiers using the same metrics as http://ranger.uta.edu/~cli/pubs/2017/claimbuster-kdd17-hassan.pdf (Table 2)

Classification report from sklearn provides everything

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import collections
import string

from sklearn.cluster import KMeans
from sklearn.metrics import *
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier

import json
import glob
import re
import time 

# Loading the data

In [2]:
df = pd.read_csv("../data_preprocessing/data.csv")
df['date'] = pd.to_datetime(df['date'])
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23462 entries, 0 to 23461
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   index       23462 non-null  int64         
 1   date        23462 non-null  datetime64[ns]
 2   Text        23462 non-null  object        
 3   Clean_text  23462 non-null  object        
 4   Verdict     23462 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 916.6+ KB


# Train-test split

In [3]:
mask = df["date"].dt.year < 2012

X_train = df.loc[mask, "Clean_text"].values
y_train = df.loc[mask, "Verdict"].values

X_test = df.loc[~mask, "Clean_text"].values
y_test = df.loc[~mask, "Verdict"].values

# Word Embedding using keras - NOT WORKING YET :D 

In [4]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [5]:
# defining vocabulary
vocabulary = {}
sentences_len = []
for sentence in X_train:
    for term in sentence.split():
        vocabulary.setdefault(term, len(vocabulary))

In [6]:
# Defining vocabulary size
vocabulary_size = list(vocabulary.values())[-1] + 1

print(f"vocabulary is composed of {vocabulary_size} unique words")

vocabulary is composed of 10205 unique words


## One hot encoding representation

Encodding train data

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)

Encodding Test data

In [8]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)

## Padding sequences

In [9]:
# finding max sentence length

vec_lengths = []
for i in X_train_encoded:
    vec_lengths.append(len(i))


max_length = np.unique(vec_lengths)[-1]


In [10]:
X_train_embedded=pad_sequences(X_train_encoded,padding='post',maxlen=max_length)
print(X_train_embedded)

[[  783   148     0 ...     0     0     0]
 [  130   110   771 ...     0     0     0]
 [  462  2841    30 ...     0     0     0]
 ...
 [    2  6525    43 ...     0     0     0]
 [ 1245    49   566 ...     0     0     0]
 [10205   264     1 ...     0     0     0]]


For training

In [11]:
X_train_embedded.shape

(18118, 65)

For testing

In [12]:
X_test_embedded=pad_sequences(X_test_encoded,padding='post',maxlen=max_length)
print(X_test_embedded.shape)

(5344, 65)


### For the labels

In [13]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(y_train.reshape(-1, 1))
y_encoded = one_hot_encoder.transform(y_train.reshape(-1, 1))

y_encoded.shape

(18118, 3)

In [14]:
y_encoded_test = one_hot_encoder.transform(y_test.reshape(-1,1))
y_encoded_test.shape

(5344, 3)

## Creating the model

In [15]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D
import tensorflow.keras.backend

In [16]:
tensorflow.keras.backend.clear_session()


In [20]:
model_conv = Sequential()
model_conv.add(Embedding(vocabulary_size+1, 100, input_length=max_length))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(100, 8, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=10))
model_conv.add(LSTM(100))
model_conv.add(Dense(32, activation = "relu"))
model_conv.add(Dense(3, activation='softmax'))
model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model_conv.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 65, 100)           1020600   
                                                                 
 dropout_1 (Dropout)         (None, 65, 100)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 58, 100)           80100     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 5, 100)           0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 32)                3232      
                                                      

In [22]:
model_conv.fit(X_train_embedded,y_encoded, validation_split=0.4, epochs = 10)

Epoch 1/10
340/340 [==============================] - 8s 19ms/step - loss: 0.7302 - accuracy: 0.7205 - val_loss: 0.6759 - val_accuracy: 0.7380
Epoch 2/10
340/340 [==============================] - 6s 19ms/step - loss: 0.5224 - accuracy: 0.8062 - val_loss: 0.6720 - val_accuracy: 0.7405
Epoch 3/10
340/340 [==============================] - 6s 19ms/step - loss: 0.3755 - accuracy: 0.8562 - val_loss: 0.7717 - val_accuracy: 0.7253
Epoch 4/10
340/340 [==============================] - 6s 19ms/step - loss: 0.2684 - accuracy: 0.8979 - val_loss: 0.8685 - val_accuracy: 0.7156
Epoch 5/10
340/340 [==============================] - 6s 19ms/step - loss: 0.1706 - accuracy: 0.9380 - val_loss: 1.1819 - val_accuracy: 0.7065
Epoch 6/10
340/340 [==============================] - 6s 19ms/step - loss: 0.1051 - accuracy: 0.9644 - val_loss: 1.1098 - val_accuracy: 0.7067
Epoch 7/10
340/340 [==============================] - 7s 20ms/step - loss: 0.0746 - accuracy: 0.9758 - val_loss: 1.3269 - val_accuracy: 0.6951

In [23]:
predictions = model_conv.predict(X_test_embedded)

In [24]:
preds = one_hot_encoder.inverse_transform(predictions).reshape(-1,)

In [25]:
preds

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

In [26]:
print(classification_report(y_test, preds, target_names=["NFS", "UFS", "CFS"]))

              precision    recall  f1-score   support

         NFS       0.75      0.87      0.80      3296
         UFS       0.28      0.28      0.28       623
         CFS       0.65      0.40      0.50      1425

    accuracy                           0.68      5344
   macro avg       0.56      0.52      0.53      5344
weighted avg       0.67      0.68      0.66      5344



In [27]:
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D
import tensorflow
tensorflow.keras.backend.clear_session()



In [28]:
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocabulary_size+1, embedding_dim, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 65, 100)           1020600   
                                                                 
 conv1d (Conv1D)             (None, 61, 128)           64128     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1,086,051
Trainable params: 1,086,051
Non-trainable params: 0
______________________________________________

In [29]:
model.fit(X_train_embedded,y_encoded, validation_split=0.2, epochs = 10)

Epoch 1/10
453/453 [==============================] - 7s 14ms/step - loss: 0.6671 - accuracy: 0.7344 - val_loss: 0.6659 - val_accuracy: 0.7252
Epoch 2/10
453/453 [==============================] - 6s 14ms/step - loss: 0.4321 - accuracy: 0.8363 - val_loss: 0.6801 - val_accuracy: 0.7290
Epoch 3/10
453/453 [==============================] - 6s 14ms/step - loss: 0.2372 - accuracy: 0.9175 - val_loss: 0.8937 - val_accuracy: 0.7243
Epoch 4/10
453/453 [==============================] - 6s 14ms/step - loss: 0.1145 - accuracy: 0.9650 - val_loss: 1.0796 - val_accuracy: 0.7081
Epoch 5/10
453/453 [==============================] - 6s 14ms/step - loss: 0.0599 - accuracy: 0.9817 - val_loss: 1.2710 - val_accuracy: 0.7058
Epoch 6/10
453/453 [==============================] - 6s 14ms/step - loss: 0.0327 - accuracy: 0.9905 - val_loss: 1.4058 - val_accuracy: 0.7047
Epoch 7/10
453/453 [==============================] - 6s 14ms/step - loss: 0.0196 - accuracy: 0.9948 - val_loss: 1.6033 - val_accuracy: 0.7086